## Fake News Classifier Using LSTM and Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [223]:
import pandas as pd

In [229]:
df=pd.read_csv('/content/train.csv', engine="python", encoding='latin-1', error_bad_lines=False)

<ipython-input-229-9fd9b59d3410>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/content/train.csv', engine="python", encoding='latin-1', error_bad_lines=False)
Skipping line 3994: unexpected end of data


In [230]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [231]:
df.label.value_counts()

1    2030
0    1962
Name: label, dtype: int64

In [232]:
df.drop('id',axis=1,inplace=True)

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3992 entries, 0 to 3991
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3886 non-null   object
 1   author  3593 non-null   object
 2   text    3983 non-null   object
 3   label   3992 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 124.9+ KB


In [234]:
df.shape

(3992, 4)

In [235]:
df.head()

,title,author,text,label
0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [236]:
df.isnull().sum()

title     106
author    399
text        9
label       0
dtype: int64

In [237]:
###Drop Nan Values
df=df.dropna()


In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3487 entries, 0 to 3991
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3487 non-null   object
 1   author  3487 non-null   object
 2   text    3487 non-null   object
 3   label   3487 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 136.2+ KB


In [239]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [240]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [241]:
## Get the Dependent features
y=df['label']

In [242]:
X.shape

(3487, 3)

In [243]:
y.shape

(3487,)

In [244]:
import tensorflow as tf

In [245]:
tf.__version__

'2.12.0'

In [246]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense

In [247]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [248]:
messages=X.copy()

In [249]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [250]:
messages

,title,author,text
0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
3987,Breaking/Exclusive: Comey Lied: FBI âSyncedâ...,"Gordon Duff, Senior Editor","By Gordon Duff, Senior Editor on October 30, 2..."
3988,Street Artist Mocks Kim Jong-Un with âTrump ...,Daniel Nussbaum,Los street artist provocateur Sabo is back a...
3989,Re: Get Ready For Civil Unrest: Survey Finds T...,Bolofia,Print \nCould we see violence no matter who wi...
3990,"As a House Burns, Tenantsâ Lives Flash Befor...",Wendy Lesser,NIGHT OF FIREBy Colin Thubron361 pp. Publish...


In [251]:
messages.reset_index(inplace=True)

In [252]:
messages

,index,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
3482,3987,Breaking/Exclusive: Comey Lied: FBI âSyncedâ...,"Gordon Duff, Senior Editor","By Gordon Duff, Senior Editor on October 30, 2..."
3483,3988,Street Artist Mocks Kim Jong-Un with âTrump ...,Daniel Nussbaum,Los street artist provocateur Sabo is back a...
3484,3989,Re: Get Ready For Civil Unrest: Survey Finds T...,Bolofia,Print \nCould we see violence no matter who wi...
3485,3990,"As a House Burns, Tenantsâ Lives Flash Befor...",Wendy Lesser,NIGHT OF FIREBy Colin Thubron361 pp. Publish...


In [253]:
import nltk
import re
from nltk.corpus import stopwords

In [254]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [255]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [256]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [257]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [258]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[705, 3812, 1676, 848, 3133, 221, 4041, 2129, 1403, 4877],
 [389, 4691, 3829, 118, 1088, 2307, 1814],
 [1437, 700, 838, 32],
 [4043, 4967, 3162, 3734, 2252, 3111],
 [3798, 1088, 2968, 2613, 1577, 4857, 1088, 876, 188, 2649],
 [2639,
  1104,
  2693,
  4636,
  2600,
  4534,
  1049,
  986,
  1425,
  4992,
  3002,
  3733,
  2975,
  3681,
  1814],
 [4924, 1621, 985, 2987, 164, 2981, 242, 2256, 638, 809, 4547],
 [1492, 437, 4430, 3876, 3843, 1612, 4534, 3546, 638, 809, 4547],
 [3159, 4809, 882, 1823, 144, 1326, 3801, 1528, 4534, 3929],
 [4591, 1327, 2908, 2289, 3434, 1962, 1913, 3953],
 [1348, 3634, 2749, 3255, 1076, 4387, 1075, 156, 4260, 1027, 1189],
 [3734, 3158, 3133, 1326, 4534, 3843],
 [4013, 4028, 955, 193, 628, 2783, 363, 1416, 3772],
 [3316, 3786, 2170, 276, 2494, 3460, 4673, 638, 809, 4547],
 [4844, 4986, 285, 2511, 4609, 638, 809, 4547],
 [538, 1255, 1982, 3656, 1573, 695, 3445, 3581, 777, 308],
 [259, 2768, 4691],
 [3205, 4773, 1531, 2056, 4534, 2406, 1830, 1814],
 [4272, 370, 3

In [259]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [260]:
onehot_repr[1]

[389, 4691, 3829, 118, 1088, 2307, 1814]

### Embedding Representation

In [261]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 705 3812 1676 ...    0    0    0]
 [ 389 4691 3829 ...    0    0    0]
 [1437  700  838 ...    0    0    0]
 ...
 [ 838 1028 3116 ...    0    0    0]
 [ 705  439 4153 ...    0    0    0]
 [4073 1330 4262 ...    0    0    0]]


In [262]:
embedded_docs[1]

array([ 389, 4691, 3829,  118, 1088, 2307, 1814,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [263]:
embedded_docs[0]

array([ 705, 3812, 1676,  848, 3133,  221, 4041, 2129, 1403, 4877,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [264]:
len(embedded_docs),y.shape

(3487, (3487,))

In [265]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [266]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [267]:
X_final.shape,y_final.shape

((3487, 20), (3487,))

In [268]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [346]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=128)

Epoch 1/10
19/19 [==============================] - 0s 27ms/step - loss: 3.2343e-05 - accuracy: 1.0000 - val_loss: 0.9797 - val_accuracy: 0.8801
Epoch 2/10
19/19 [==============================] - 0s 23ms/step - loss: 3.7857e-04 - accuracy: 0.9996 - val_loss: 0.8612 - val_accuracy: 0.8836
Epoch 3/10
19/19 [==============================] - 0s 11ms/step - loss: 6.9047e-05 - accuracy: 1.0000 - val_loss: 0.8816 - val_accuracy: 0.8827
Epoch 4/10
19/19 [==============================] - 1s 30ms/step - loss: 1.7951e-05 - accuracy: 1.0000 - val_loss: 0.8952 - val_accuracy: 0.8827
Epoch 5/10
19/19 [==============================] - 1s 35ms/step - loss: 1.3145e-05 - accuracy: 1.0000 - val_loss: 0.8996 - val_accuracy: 0.8827
Epoch 6/10
19/19 [==============================] - 1s 38ms/step - loss: 1.2281e-05 - accuracy: 1.0000 - val_loss: 0.9023 - val_accuracy: 0.8827
Epoch 7/10
19/19 [==============================] - 0s 19ms/step - loss: 1.1889e-05 - accuracy: 1.0000 - val_loss: 0.9058 - val_ac

### Performance Metrics And Accuracy

In [347]:
y_pred=model.predict(X_test)

36/36 [==============================] - 0s 3ms/step


In [348]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [349]:
from sklearn.metrics import confusion_matrix

In [350]:
confusion_matrix(y_test,y_pred)

array([[582,  65],
       [ 69, 435]])

In [351]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8835794960903562

In [352]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90       647
           1       0.87      0.86      0.87       504

    accuracy                           0.88      1151
   macro avg       0.88      0.88      0.88      1151
weighted avg       0.88      0.88      0.88      1151



In [306]:
## Creating model Bidirectional LSTM
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 20, 40)            200000    
                                                                 
 dropout_12 (Dropout)        (None, 20, 40)            0         
                                                                 
 bidirectional_9 (Bidirectio  (None, 256)              173056    
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 257       
                                                                 
Total params: 373,313
Trainable params: 373,313
Non-trainable params: 0
_______________________________________________

In [307]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [308]:
X_final.shape,y_final.shape

((3487, 20), (3487,))

In [309]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [340]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=128)

Epoch 1/10
19/19 [==============================] - 4s 204ms/step - loss: 1.2371e-05 - accuracy: 1.0000 - val_loss: 0.9396 - val_accuracy: 0.8818
Epoch 2/10
19/19 [==============================] - 3s 183ms/step - loss: 1.0502e-05 - accuracy: 1.0000 - val_loss: 0.9412 - val_accuracy: 0.8818
Epoch 3/10
19/19 [==============================] - 2s 126ms/step - loss: 1.6304e-05 - accuracy: 1.0000 - val_loss: 0.9406 - val_accuracy: 0.8818
Epoch 4/10
19/19 [==============================] - 2s 105ms/step - loss: 1.1570e-05 - accuracy: 1.0000 - val_loss: 0.9419 - val_accuracy: 0.8818
Epoch 5/10
19/19 [==============================] - 2s 125ms/step - loss: 1.5675e-05 - accuracy: 1.0000 - val_loss: 0.9444 - val_accuracy: 0.8818
Epoch 6/10
19/19 [==============================] - 1s 58ms/step - loss: 1.0386e-05 - accuracy: 1.0000 - val_loss: 0.9468 - val_accuracy: 0.8818
Epoch 7/10
19/19 [==============================] - 1s 71ms/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.9592 - val_a

In [341]:
y_pred=model.predict(X_test)

36/36 [==============================] - 0s 3ms/step


In [342]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [343]:
confusion_matrix(y_test,y_pred)

array([[591,  56],
       [ 83, 421]])

In [344]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8792354474370113

In [345]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       647
           1       0.88      0.84      0.86       504

    accuracy                           0.88      1151
   macro avg       0.88      0.87      0.88      1151
weighted avg       0.88      0.88      0.88      1151

